<a href="https://colab.research.google.com/github/microsoft/FLAML/blob/main/notebook/autogen_agent_MathChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using MathChat to Solve Math Problems

MathChat is a convesational framework for math problem solving. In this notebook, we demonstrate how to use MathChat to solve math problems. MathChat uses the `AssistantAgent` and `MathUserProxyAgent`, which is similar to the usage of `AssistantAgent` and `UserProxyAgent` in other notebooks (More details in `autogen_agent_auto_feedback_from_code_execution.ipynb`). Essentially, `MathUserProxyAgent` implements a different auto reply mechanism corresponding to the MathChat prompts. The original implementation and exeperiments of MathChat are in this [branch](https://github.com/kevin666aa/FLAML/tree/gpt_math_solver/flaml/autogen/math), and you can find more details in our paper [An Empirical Study on Challenging Math Problem Solving with GPT-4](https://arxiv.org/abs/2306.01337).

## Requirements

FLAML requires `Python>=3.7`. To run this notebook example, please install flaml with the [mathchat] option.
```bash
pip install flaml[mathchat]
```

In [ ]:
# %pip install flaml[mathchat]~=2.0.0rc4

## Set your API Endpoint

The [`config_list_gpt4_gpt35`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_gpt4_gpt35) function tries to create a list of gpt-4 and gpt-3.5 configurations using Azure OpenAI endpoints and OpenAI endpoints. It assumes the api keys and api bases are stored in the corresponding environment variables or local txt files:

- OpenAI API key: os.environ["OPENAI_API_KEY"] or `openai_api_key_file="key_openai.txt"`.
- Azure OpenAI API key: os.environ["AZURE_OPENAI_API_KEY"] or `aoai_api_key_file="key_aoai.txt"`. Multiple keys can be stored, one per line.
- Azure OpenAI API base: os.environ["AZURE_OPENAI_API_BASE"] or `aoai_api_base_file="base_aoai.txt"`. Multiple bases can be stored, one per line.

It's OK to have only the OpenAI API key, or only the Azure OpenAI API key + base.

If you open this notebook in google colab, you can upload your files by click the file icon on the left panel and then choose "upload file" icon.


In [1]:
from flaml import oai

config_list = oai.config_list_gpt4_gpt35()

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },  # only if OpenAI API key is found
    {
        'model': 'gpt-4',
        'api_key': '<your first Azure OpenAI API key here>',
        'api_base': '<your first Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-03-15-preview',
    },  # only if the at least one Azure OpenAI API key is found
    {
        'model': 'gpt-4',
        'api_key': '<your second Azure OpenAI API key here>',
        'api_base': '<your second Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-03-15-preview',
    },  # only if the second Azure OpenAI API key is found
]
```

You can directly override it if the above function returns an empty list, i.e., it doesn't find the keys in the specified locations.

## Construct agents for MathChat

We start by initialzing the `AssistantAgent` and `MathUserProxyAgent`. The system message needs to be set to "You are a helpful assistant." for MathChat. The detailed instructions are given in the user message. Later we will use the `MathUserProxyAgent.generate_init_prompt` to combine the instructions and a math problem for an initial prompt to be sent to the LLM assistant.

In [11]:
from flaml.autogen.agent.assistant_agent import AssistantAgent
from flaml.autogen.agent.math_user_proxy_agent import MathUserProxyAgent

oai.ChatCompletion.start_logging()

# 1. create an AssistantAgent instance named "assistant"
assistant = AssistantAgent(
    name="assistant", 
    system_message="You are a helpful assistant.",
    request_timeout=600, 
    seed=42, 
    config_list=config_list,
)

# 2. create the MathUserProxyAgent instance named "mathproxyagent"
# By default, the human_input_mode is "NEVER", which means the agent will not ask for human input.
mathproxyagent = MathUserProxyAgent(
    name="MathChatAgent", 
    human_input_mode="NEVER",
    use_docker=False,
)

### Example 1

Problem: Find all $x$ that satisfy the inequality $(2x+10)(x+3)<(3x+9)(x+8)$. Express your answer in interval notation.

Correct Solution: 
We have \begin{align*} (2x+10)(x+3)&<(3x+9)(x+8) \quad \Rightarrow
\\ 2(x+5)(x+3)&<3(x+3)(x+8) \quad \Rightarrow
\\ 2(x+5)(x+3)-3(x+3)(x+8)&<0 \quad \Rightarrow
\\ (2x+10-(3x+24))(x+3)&<0 \quad \Rightarrow
\\ (-x-14)(x+3)&<0 \quad \Rightarrow
\\ (x+14)(x+3)&>0.
\end{align*} This inequality is satisfied if and only if $(x+14)$ and $(x+3)$ are either both positive or both negative.  Both factors are positive for $x>-3$ and both factors are negative for $x<-14$.  When $-14<x<-3$, one factor is positive and the other negative, so their product is negative.   Therefore, the range of $x$ that satisfies the inequality is $ \boxed{(-\infty, -14)\cup(-3,\infty)} $.

In [9]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# given a math problem, we use the mathproxyagent to generate a prompt to be sent to the assistant as the initial message.
# the assistant receives the message and generates a response. The response will be sent back to the mathproxyagent for processing.
# The conversation continues until the termination condition is met, in MathChat, the termination condition is the detect of "\boxed{}" in the response.
math_problem = "Find all $x$ that satisfy the inequality $(2x+10)(x+3)<(3x+9)(x+8)$. Express your answer in interval notation."
mathproxyagent.initiate_chat(assistant, math_problem)


 -------------------------------------------------------------------------------- 

MathChatAgent (to assistant):
Let's use Python to solve a math problem.

Query requirements:
You should always use the 'print' function for the output and use fractions/radical forms instead of decimals.
You can use packages like sympy to help you.
You must follow the formats below to write your code:
```python
# your code
```

First state the key idea to solve the problem. You may choose from three ways to solve the problem:
Case 1: If the problem can be solved with Python code directly, please write a program to solve it. You can enumerate all possible arrangements if needed.
Case 2: If the problem is mostly reasoning, you can solve it by yourself directly.
Case 3: If the problem cannot be handled in the above two ways, please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python (for example, any calculations o

### Example 2
Problem: For what negative value of $k$ is there exactly one solution to the system of equations \begin{align*}
y &= 2x^2 + kx + 6 \\
y &= -x + 4?
\end{align*}

Correct Solution: Setting the two expressions for $y$ equal to each other, it follows that $2x^2 + kx + 6 = -x + 4$. Re-arranging, $2x^2 + (k+1)x + 2 = 0$. For there to be exactly one solution for $x$, then the discriminant of the given quadratic must be equal to zero. Thus, $(k+1)^2 - 4 \cdot 2 \cdot 2 = (k+1)^2 - 16 = 0$, so $k+1 = \pm 4$. Taking the negative value, $k = \boxed{-5}$.

In [27]:
assistant.reset()

math_problem = "For what negative value of $k$ is there exactly one solution to the system of equations \\begin{align*}\ny &= 2x^2 + kx + 6 \\\\\ny &= -x + 4?\n\\end{align*}"
mathproxyagent.initiate_chat(assistant, math_problem)


**** assistant received message from MathChatAgent ****


Let's use Python to solve a math problem.

Query requirements:
You should always use the 'print' function for the output and use fractions/radical forms instead of decimals.
You can use packages like sympy to help you.
You must follow the formats below to write your code:
```python
# your code
```

You may choose from three ways to solve the problem:
Case 1: If the problem can be solved with Python code directly, please write a program to solve it. You can enumerate all possible arrangements if needed.
Case 2: If the problem cannot be handled in the above two ways, please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python (for example, any calculations or equations that can be calculated).
3. Wait for me to give the results.
4. Continue if you think the result is correct. If the result is invalid or unexpected, please correct your query

### Example 3
Problem: Find all positive integer values of $c$ such that the equation $x^2-7x+c=0$ only has roots that are real and rational. Express them in decreasing order, separated by commas.

Correct Solution: For the roots to be real and rational, the discriminant must be a perfect square. Therefore, $(-7)^2-4 \cdot 1 \cdot c = 49-4c$ must be a perfect square. The only positive perfect squares less than 49 are $1$, $4$, $9$, $16$, $25$, and $36$. The perfect squares that give a integer value of $c$ are $1$, $9$, and $25$. Thus, we have the equations $49-4c=1$, $49-4c=9$, and $49-4c=25$. Solving, we get that the positive integer values of c are $\boxed{12, 10, 6}$.

In [30]:
assistant.reset()

math_problem = "Find all positive integer values of $c$ such that the equation $x^2-7x+c=0$ only has roots that are real and rational. Express them in decreasing order, separated by commas."
mathproxyagent.initiate_chat(assistant, math_problem)


**** assistant received message from MathChatAgent ****


Let's use Python to solve a math problem.

Query requirements:
You should always use the 'print' function for the output and use fractions/radical forms instead of decimals.
You can use packages like sympy to help you.
You must follow the formats below to write your code:
```python
# your code
```

You may choose from three ways to solve the problem:
Case 1: If the problem can be solved with Python code directly, please write a program to solve it. You can enumerate all possible arrangements if needed.
Case 2: If the problem cannot be handled in the above two ways, please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python (for example, any calculations or equations that can be calculated).
3. Wait for me to give the results.
4. Continue if you think the result is correct. If the result is invalid or unexpected, please correct your query

------------------------------------
### Using other prompts

MathChat allows different prompts that instruct assistant to solve the problem.

Check out `MathUserProxyAgent.generate_init_prompt(problem, prompt_type='default', customized_prompt=None)`:
- You may choose from `['default', 'python', 'two_tools']` for parameter `prompt_type`.  We include two more prompts in the paper: 
    1. `'python'` is a simplified prompt from the default prompt that uses Python only. 
    2. `'two_tools'` further allows the selection of Python or Wolfram Alpha based on this simplified `python` prompt. Note that this option requries a Wolfram Alpha API key and put it in `wolfram.txt`.

- You can also input your customized prompt if needed: `mathproxyagent.generate_init_prompt(problem, customized_prompt="Your customized prompt")`. Since this mathproxyagent detects '\boxed{}' as termination, you need to have a similar termination sentence in the prompt: "If you get the answer, put the answer in \\boxed{}.". If the customized is provided, the `prompt_type` will be ignored.


### Example 4 (Use the "python" prompt):

Problem: If $725x + 727y = 1500$ and $729x+ 731y = 1508$, what is the value of $x - y$ ?

Correct Solution: Subtracting the two equations gives: 
\begin{align*}
(729x+731y)-(725x+727y) &= 1508-1500\\
\Rightarrow\qquad 4x+4y &= 8\\
\Rightarrow\qquad x+y &= 2.
\end{align*}

Multiplying this equation by 725 and subtracting this equation from $725x+727y=1500$ gives \begin{align*}
(725x+727y) - 725(x+y) &= 1500-725(x+y) \implies \\
2y &= 50.
\end{align*}So we can write $x-y$ as $(x+y) - 2y$, which equals  $2 - 50 = \boxed{-48}$.


In [13]:
assistant.reset() # clear LLM assistant's message history

# we set the prompt_type to "python", which is a simplied version of the default prompt.
math_problem = "Problem: If $725x + 727y = 1500$ and $729x+ 731y = 1508$, what is the value of $x - y$ ?"
mathproxyagent.initiate_chat(assistant, math_problem, prompt_type="python")


 -------------------------------------------------------------------------------- 

MathChatAgent (to assistant):
Let's use Python to solve a math problem.

Query requirements:
You should always use the 'print' function for the output and use fractions/radical forms instead of decimals.
You can use packages like sympy to help you.
You must follow the formats below to write your code:
```python
# your code
```

Please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python (for example, any calculations or equations that can be calculated).
3. Wait for me to give the results.
4. Continue if you think the result is correct. If the result is invalid or unexpected, please correct your query or reasoning.

After all the queries are run and you get the answer, put the answer in \boxed{}.

Problem: Problem: If $725x + 727y = 1500$ and $729x+ 731y = 1508$, what is the value of $x - y$ ?

 -----------------

## Example 5 (Use the "two_tools" prompt)

Problem: Find all numbers $a$ for which the graph of $y=x^2+a$ and the graph of $y=ax$ intersect. Express your answer in interval notation.


Correct Solution: If these two graphs intersect then the points of intersection occur when  \[x^2+a=ax,\] or  \[x^2-ax+a=0.\] This quadratic has solutions exactly when the discriminant is nonnegative: \[(-a)^2-4\cdot1\cdot a\geq0.\] This simplifies to  \[a(a-4)\geq0.\] This quadratic (in $a$) is nonnegative when $a$ and $a-4$ are either both $\ge 0$ or both $\le 0$. This is true for $a$ in $$(-\infty,0]\cup[4,\infty).$$ Therefore the line and quadratic intersect exactly when $a$ is in $\boxed{(-\infty,0]\cup[4,\infty)}$.


In [15]:
assistant.reset() # clear LLM assistant's message history

# The wolfram alpha appid is required for this example (the assistant may choose to query Wolfram Alpha).
import os
if "WOLFRAM_ALPHA_APPID" not in os.environ:
    os.environ["WOLFRAM_ALPHA_APPID"] = open("wolfram.txt").read().strip()

# we set the prompt_type to "two_tools", which allows the assistant to select wolfram alpha when necessary.
math_problem = "Find all numbers $a$ for which the graph of $y=x^2+a$ and the graph of $y=ax$ intersect. Express your answer in interval notation."
mathproxyagent.initiate_chat(assistant, math_problem, prompt_type="two_tools")


 -------------------------------------------------------------------------------- 

MathChatAgent (to assistant):
Let's use two tools (Python and Wolfram alpha) to solve a math problem.

Query requirements:
You must follow the formats below to write your query:
For Wolfram Alpha:
```wolfram
# one wolfram query
```
For Python:
```python
# your code
```
When using Python, you should always use the 'print' function for the output and use fractions/radical forms instead of decimals. You can use packages like sympy to help you.
When using wolfram, give one query in each code block.

Please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python or Wolfram Alpha, select the most suitable tool to be used (for example, any calculations or equations that can be calculated).
3. Wait for me to give the results.
4. Continue if you think the result is correct. If the result is invalid or unexpected, please corr